In [2]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium beautifulsoup4 webdriver_manager

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,153 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,308 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:14 http

# Scrapping

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")


# Navigate to the BBC Weather page for Tel Aviv
url = "https://www.bbc.com/weather/293397"
driver.get(url)

# Wait for the weather data to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "wr-day__weather-type-description")))

# Get the page source and parse it with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find all day elements
days = soup.find_all('li', class_='wr-day')

# Initialize lists to store data
dates = []
max_temps = []
min_temps = []
weather_types = []

# Extract data from each day
for day in days:
    date = day.find('span', class_='wr-date__long').text.strip()
    max_temp = day.find('span', class_='wr-value--temperature--c').text.strip()
    min_temp = day.find('span', class_='wr-temperature__low').text.strip()
    weather_type = day.find('span', class_='wr-day__weather-type-description').text.strip()

    dates.append(date)
    max_temps.append(max_temp)
    min_temps.append(min_temp)
    weather_types.append(weather_type)

# Create a DataFrame
df = pd.DataFrame({
    'Date': dates,
    'Max Temp (°C)': max_temps,
    'Min Temp (°C)': min_temps,
    'Weather Type': weather_types
})

# Clean the data
df['Max Temp (°C)'] = df['Max Temp (°C)'].str.replace('°', '').astype(int)
df['Min Temp (°C)'] = df['Min Temp (°C)'].str.replace('°', '').astype(int)

# Save to CSV
df.to_csv('tel_aviv_weather.csv', index=False)

# Close the browser
driver.quit()

print(df)

NameError: name 'driver' is not defined

# Visualizations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv('tel_aviv_weather.csv')

# Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Calculate average temperature
df['Avg Temp (°C)'] = (df['Max Temp (°C)'] + df['Min Temp (°C)']) / 2

# Basic analysis
avg_max_temp = df['Max Temp (°C)'].mean()
avg_min_temp = df['Min Temp (°C)'].mean()
most_common_weather = df['Weather Type'].mode().values[0]

print(f"Average Max Temperature: {avg_max_temp:.1f}°C")
print(f"Average Min Temperature: {avg_min_temp:.1f}°C")
print(f"Most common weather type: {most_common_weather}")

# Visualization 1: Temperature trends
plt.figure(figsize=(12, 6))a
plt.plot(df['Date'], df['Max Temp (°C)'], label='Max Temp')
plt.plot(df['Date'], df['Min Temp (°C)'], label='Min Temp')
plt.plot(df['Date'], df['Avg Temp (°C)'], label='Avg Temp')
plt.title('Temperature Trends in Tel Aviv')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('temperature_trends.png')
plt.close()

# Visualization 2: Weather type distribution
plt.figure(figsize=(10, 6))
sns.countplot(y='Weather Type', data=df, order=df['Weather Type'].value_counts().index)
plt.title('Distribution of Weather Types')
plt.xlabel('Count')
plt.ylabel('Weather Type')
plt.tight_layout()
plt.savefig('weather_type_distribution.png')
plt.close()

# Visualization 3: Temperature range
plt.figure(figsize=(12, 6))
sns.boxplot(x='Date', y='value', hue='variable',
            data=pd.melt(df[['Date', 'Max Temp (°C)', 'Min Temp (°C)']], ['Date']))
plt.title('Temperature Range in Tel Aviv')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.xticks(rotation=45)
plt.legend(title='')
plt.tight_layout()
plt.savefig('temperature_range.png')
plt.close()

print("Analysis complete. Visualizations saved as PNG files.")